# Bayesian Model Mixing and Optimization
### Evan Edwards

- Multivariate Mixing With The Following Models That Use Mixed-Informative Priors:
    - PV1READ ~ Female + ESCS + HOMEPOS + ICTRES + (1 + ICTRES | SchoolID)
    - PV1READ ~ JOYREAD + PISADIFF + SCREADCOMP + SCREADDIFF + (1|SchoolID)
    - PV1READ ~ METASUM + GFOFAIL + MASTGOAL + SWBP + WORKMAST + ADAPTIVITY + COMPETE + (1|SchoolID)
    - PV1READ ~ PERFEED + TEACHINT + BELONG + (1 + TEACHINT | SchoolID)
- Sensitivity Analysis
- Optimization

In [1]:
from Taweret.core.base_model import BaseModel
import numpy as np
import pymc as pm
import pandas as pd
from sklearn.metrics import mean_squared_error
from Taweret.core.base_mixer import BaseMixer
from math import sqrt
from scipy.optimize import minimize
import logging
import bambi as bmb

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#Fixed random seed to ensure reproducibility and the possiblility for optimization
RANDOM_SEED = 9572404
rng = np.random.default_rng(RANDOM_SEED)

In [3]:
# Defining to disable output later for ease of visibility
logger = logging.getLogger("pymc")
# Disabling sampling messages
logger.setLevel(logging.ERROR)

In [4]:
# Load dataset
PISA2018 = pd.read_csv("pisa2018.BayesBook.csv")
# Data processing: converting categorical values to numerical values
PISA2018['Female'] = PISA2018['Female'].replace({'Female': 1.0, 'Male': 0.0})
# Converting numerical to categorical values
PISA2018['SchoolID'] = pd.Categorical(PISA2018['SchoolID']).codes

In [5]:
%%time
#PV1READ ~ Female + ESCS + HOMEPOS + ICTRES + (1 + ICTRES | SchoolID)
model1 = bmb.Model("PV1READ ~ Female + ESCS + HOMEPOS + ICTRES + (1 + ICTRES | SchoolID)", PISA2018, categorical = ["SchoolID"])
priors = {"Intercept": bmb.Prior("Normal", mu=0, sigma=100),
          "Female": bmb.Prior("Normal", mu=0, sigma=10),
          "ESCS": bmb.Prior("Normal", mu=np.mean(PISA2018["ESCS"]), sigma=np.std(PISA2018["ESCS"])),
          "HOMEPOS": bmb.Prior("Normal", mu=np.mean(PISA2018["HOMEPOS"]), sigma=100),
          "ICTRES": bmb.Prior("Normal", mu=np.mean(PISA2018["ICTRES"]), sigma=np.std(PISA2018["ICTRES"])),
          "1|SchoolID": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfNormal", sigma=100)),
          "ICTRES|SchoolID": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfNormal", sigma=100)),
          "sigma": bmb.Prior("HalfNormal", sigma=10)}
model1.set_priors(priors = priors)

trace1 = model1.fit(draws=2000, random_seed=RANDOM_SEED)

post_pred = model1.predict(trace1,data = PISA2018, inplace=False).posterior["PV1READ_mean"]
mean_pred = np.array(post_pred.mean(dim=["chain", "draw"]))
print(f'The RMSE for model 1 - PV1READ ~ Female + ESCS + HOMEPOS + ICTRES + (1 + ICTRES | SchoolID) is: {sqrt(mean_squared_error(PISA2018["PV1READ"], mean_pred))}')

The RMSE for model 1 - PV1READ ~ Female + ESCS + HOMEPOS + ICTRES + (1 + ICTRES | SchoolID) is: 94.1648602282953
CPU times: total: 56.7 s
Wall time: 3min 36s


In [6]:
%%time
#PV1READ ~ JOYREAD + PISADIFF + SCREADCOMP + SCREADDIFF + (1|SchoolID)
model2 = bmb.Model("PV1READ ~ JOYREAD + PISADIFF + SCREADCOMP + SCREADDIFF + (1|SchoolID)", PISA2018, categorical = ["SchoolID"])

priors = {"Intercept": bmb.Prior("Normal", mu=0, sigma=100),
          "JOYREAD": bmb.Prior("Normal", mu=np.mean(PISA2018["JOYREAD"]), sigma=np.std(PISA2018["JOYREAD"])),
          "PISADIFF": bmb.Prior("Normal", mu=0, sigma=100),
          "SCREADCOMP": bmb.Prior("Normal", mu=np.mean(PISA2018["SCREADCOMP"]), sigma=10),
          "SCREADDIFF": bmb.Prior("Normal", mu=np.mean(PISA2018["SCREADDIFF"]), sigma=np.std(PISA2018["SCREADDIFF"])),
          "1|SchoolID": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfNormal", sigma=100)),
          "sigma": bmb.Prior("HalfNormal", sigma=10)}
model2.set_priors(priors = priors)

trace2 = model2.fit(draws=2000, random_seed=RANDOM_SEED)

post_pred = model2.predict(trace2, data = PISA2018, inplace=False).posterior["PV1READ_mean"]
mean_pred = np.array(post_pred.mean(dim=["chain", "draw"]))
print(f'The RMSE for model 2 - PV1READ ~ JOYREAD + PISADIFF + SCREADCOMP + SCREADDIFF + (1|SchoolID) is: {sqrt(mean_squared_error(PISA2018["PV1READ"], mean_pred))}')

The RMSE for model 2 - PV1READ ~ JOYREAD + PISADIFF + SCREADCOMP + SCREADDIFF + (1|SchoolID) is: 86.0274037911334
CPU times: total: 44.5 s
Wall time: 3min 20s


In [7]:
%%time
#PV1READ ~ METASUM + GFOFAIL + MASTGOAL + SWBP + WORKMAST + ADAPTIVITY + COMPETE + (1|SchoolID)
model3 = bmb.Model("PV1READ ~ METASUM + GFOFAIL + MASTGOAL + SWBP + WORKMAST + ADAPTIVITY + COMPETE + (1|SchoolID)", PISA2018, categorical = ["SchoolID"])

priors = {"Intercept": bmb.Prior("Normal", mu=0, sigma=100),
          "METASUM": bmb.Prior("Normal", mu=np.mean(PISA2018["METASUM"]), sigma=np.std(PISA2018["METASUM"])),
          "GFOFAIL": bmb.Prior("Normal", mu=0, sigma=100),
          "MASTGOAL": bmb.Prior("Normal", mu=np.mean(PISA2018["MASTGOAL"]), sigma=10),
          "SWBP": bmb.Prior("Normal", mu=0, sigma=100),
          "WORKMAST": bmb.Prior("Normal", mu=np.mean(PISA2018["WORKMAST"]), sigma=10),
          "ADAPTIVITY": bmb.Prior("Normal", mu=np.mean(PISA2018["ADAPTIVITY"]), sigma=100),
          "COMPETE": bmb.Prior("Normal", mu=np.mean(PISA2018["COMPETE"]), sigma=np.std(PISA2018["COMPETE"])),
          "1|SchoolID": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfNormal", sigma=100)),
          "sigma": bmb.Prior("HalfNormal", sigma=10)}
model3.set_priors(priors = priors)

trace3 = model3.fit(draws=2000, random_seed=RANDOM_SEED)

post_pred = model3.predict(trace3, data = PISA2018, inplace=False).posterior["PV1READ_mean"]
mean_pred = np.array(post_pred.mean(dim=["chain", "draw"]))
print(f'The RMSE for model 3 - PV1READ ~ METASUM + GFOFAIL + MASTGOAL + SWBP + WORKMAST + ADAPTIVITY + COMPETE + (1|SchoolID) is: {sqrt(mean_squared_error(PISA2018["PV1READ"], mean_pred))}')

The RMSE for model 3 - PV1READ ~ METASUM + GFOFAIL + MASTGOAL + SWBP + WORKMAST + ADAPTIVITY + COMPETE + (1|SchoolID) is: 91.30095525810611
CPU times: total: 1min 1s
Wall time: 4min 50s


In [8]:
%%time
#PV1READ ~ PERFEED + TEACHINT + BELONG + (1 + TEACHINT | SchoolID)
model4 = bmb.Model("PV1READ ~ PERFEED + TEACHINT + BELONG + (1 + TEACHINT | SchoolID)", PISA2018, categorical = ["SchoolID"])

priors = {"Intercept": bmb.Prior("Normal", mu=0, sigma=100),
          "PERFEED": bmb.Prior("Normal", mu=np.mean(PISA2018["PERFEED"]), sigma=np.std(PISA2018["PERFEED"])),
          "TEACHINT": bmb.Prior("Normal", mu=np.mean(PISA2018["TEACHINT"]), sigma=np.std(PISA2018["TEACHINT"])),
          "BELONG": bmb.Prior("Normal", mu=np.mean(PISA2018["BELONG"]), sigma=100),
          "1|SchoolID": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfNormal", sigma=100)),
          "TEACHINT|SchoolID": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfNormal", sigma=100)),
          "sigma": bmb.Prior("HalfNormal", sigma=10)}
model4.set_priors(priors = priors)

trace4 = model4.fit(draws=2000, random_seed=RANDOM_SEED)

post_pred = model4.predict(trace4, data = PISA2018, inplace=False).posterior["PV1READ_mean"]
mean_pred = np.array(post_pred.mean(dim=["chain", "draw"]))
print(f'The RMSE for model 4 - PV1READ ~ PERFEED + TEACHINT + BELONG + (1 + TEACHINT | SchoolID) is: {sqrt(mean_squared_error(PISA2018["PV1READ"], mean_pred))}')

The RMSE for model 4 - PV1READ ~ PERFEED + TEACHINT + BELONG + (1 + TEACHINT | SchoolID) is: 94.72730464194656
CPU times: total: 1min 8s
Wall time: 7min 8s


In [9]:
# A wrapper class for the Bambi/PYMC models to be compatible with the Taweret framework
class BMBWrapper(BaseModel):
    def __init__(self, model, idata, posterior_predictive):
        self.model = model
        self.idata = idata
        self.posterior_predictive = posterior_predictive
        
    def evaluate(self, model_parameters):
        post_pred = self.model.predict(self.idata, data = model_parameters, inplace=False).posterior[self.posterior_predictive]
        return np.array(post_pred.mean(dim=["chain", "draw"])).reshape(-1, 1), np.sqrt(np.array(post_pred.var(dim=["chain", "draw"]))).flatten().reshape(-1, 1)

    
    def log_likelihood_elementwise(self,x_exp, y_exp, y_err, model_params):
        y = self.evaluate(model_params)[0]
        
        return np.exp(-(y - y_exp) **2 / (2 * y_err ** 2)) \
            / np.sqrt(2 * np.pi * y_err ** 2)
    
    def set_prior(self, prior_dict):
        self.model.set_priors(self, priors=prior_dict)


In [10]:
# A rudimentary mixer class that can assign weights to the models and optimize them, a child class of BaseMixer
# Many abstract methods need to be defined, however they are not implemented becuase they are not needed

class WeightableMultivariate(BaseMixer):
    models = {}
    weights = []
    
    def __init__(self, weights, models):
        self.weights = weights
        self.models = models


    def set_params(self, weights):
        self.weights = weights

        return self


    def predict(self, X):
        cumulative_predictions = 0
        
        for x in range(0,len(self.weights)):
            model_prediction, _ = self.models[str(x+1)].evaluate(X)
            cumulative_predictions += model_prediction * self.weights[x]
        
        return cumulative_predictions
    

    def train(self):

        # The objective function to be minimized
        def objective(params):
            self.set_params(params)
            predictions = self.predict(PISA2018)
            rmse = sqrt(mean_squared_error(PISA2018["PV1READ"], predictions))
            return rmse

        # Initial guess of the weights and the constraints
        initial_params = [0.25, 0.25, 0.25, 0.25]
        constraints = ({'type': 'eq', 'fun': lambda x : np.sum(x) - 1.0})
        bounds = [(0,1),(0,1),(0,1),(0,1)]

        result = minimize(objective, initial_params, constraints=constraints, bounds = bounds)
        
        self.weights = result.x

        return self


    def evaluate_weights(self):
        return self.weights
    
    def set_prior(self):
        pass

    def prior_predict(self):
        pass
    
    def prior(self):
        pass

    def predict_weights(self):
        pass

    def posterior(self):
        pass

    def map(Self):
        pass

    def evaluate(self):
        pass

In [11]:
# Defining an instance of the custom mixer class

mixer = WeightableMultivariate([0.25, 0.25, 0.25, 0.25],
        {   "1": BMBWrapper(model1, trace1, "PV1READ_mean"),
            "2": BMBWrapper(model2, trace2, "PV1READ_mean"),
            "3": BMBWrapper(model3, trace3, "PV1READ_mean"),
            "4": BMBWrapper(model4, trace4, "PV1READ_mean"),})

In [12]:
# Basic sensitivity analysis, iterating through a set of predefined weights

for x in [[0.25, 0.25, 0.25, 0.25],
          [0.05, 0.6, 0.3, 0.05],
          [0.4, 0.15, 0.05, 0.4],
          [0.1, 0.35, 0.2, 0.35],
          [0.1, 0.2, 0.3, 0.4],
          [0.1, 0.2, 0.5, 0.2]]:
    
    mixer.set_params(x)
    
    print(f'The RMSE score of params {mixer.weights}: {((sqrt(mean_squared_error(PISA2018["PV1READ"], mixer.predict(PISA2018)))))}')

The RMSE score of params [0.25, 0.25, 0.25, 0.25]: 88.52103508972874
The RMSE score of params [0.05, 0.6, 0.3, 0.05]: 85.32236613324066
The RMSE score of params [0.4, 0.15, 0.05, 0.4]: 90.77713454677158
The RMSE score of params [0.1, 0.35, 0.2, 0.35]: 87.74482722822798
The RMSE score of params [0.1, 0.2, 0.3, 0.4]: 89.24281205118932
The RMSE score of params [0.1, 0.2, 0.5, 0.2]: 88.55136244962752


In [13]:
 # Finding optimized parameters
mixer.train()

print(f'The RMSE score of the optimized parameters {mixer.evaluate_weights()}: {((sqrt(mean_squared_error(PISA2018["PV1READ"], mixer.predict(PISA2018)))))}')

The RMSE score of the optimized parameters [2.99184452e-16 7.10967259e-01 2.89032741e-01 1.07864118e-15]: 84.9444620826476


[1.85200173e-16 7.12366185e-01 2.87633815e-01 1.65963739e-15]: 85.01453016147077

[1.85200173e-16 7.12366185e-01 2.87633815e-01 1.65963739e-15]: 85.01453016147077